In [151]:
import numpy as np
import pandas as pd
import re
import ast
import math
!pip install import-ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2979 sha256=405bdcb4d2a2505e2c94f290f77a9012931ab22239927e84f1d7b35ed79fefd7
  Stored in directory: c:\users\rames\appdata\local\pip\cache\wheels\06\7e\ad\1cb03e935234186825cefc7e2c8f3451b4f654b5bc72232a7b
Successfully built import-ipynb


In [148]:
class EntropyBasedItemset:
    def __init__(self, master):
        self.master = master
    
    def getSubsetNotNA(self, df, columnName):
        data = df.copy()
        data = data[data[columnName].notna()]
        return data
    
    def CombineCourseGrade(self, data1, column1, column2):
        data = data1.copy()
        data['CourseGradeCombo'] = ""
        data[column1] = data[column1].apply(ast.literal_eval)
        data[column2] = data[column2].apply(ast.literal_eval)
        for idx, row in data.iterrows():
            courseList = row[column1]
            gradeList = row[column2]
            #print(type(courseList))
            combo = dict(zip(courseList, gradeList))
            #print(combo)
            #data.set_value(idx, 'CourseGradeCombo', combo)
            data.at[idx,'CourseGradeCombo']=combo
        return(data)
    
    def SplitCourseToIndividualRows(self, data, columnName):
        df = pd.DataFrame([[i, k, v] for i, d in data[[columnName,'CourseGradeCombo']].values for k, v in d.items()],
                  columns=[columnName,'CourseIndivName','CourseIndivGrade'])
        return df
    
    def preprocessCourseNameGradeIndiv(self, data):
        df = data.copy()
        df.CourseIndivGrade = df.CourseIndivGrade.str.replace("\"",'')
        df.CourseIndivName = df.CourseIndivName.str.strip().str.lower().str.replace("-i"," - i")
        df.CourseIndivName = df.CourseIndivName.str.replace("web technologies 1", "web technologies - i")
        df.CourseIndivName = df.CourseIndivName.str.replace("technology", "technologies")
        df.CourseIndivName = df.CourseIndivName.str.replace("web technologies- i", "web technologies - i")
        df.CourseIndivName = df.CourseIndivName.str.replace("web technologies ii", "web technologies - ii")
        return df
    
    def setMapGradeToNumber(self, mapDict):
        mapDict = {"S": "7", "A":"6", "B":"5" ,"C":"4", "D":"3", "E":"2", "F":"1", "N":"0"}
        self.gradeMap = mapDict
    
    def mapColumnGradeToNumber(self, df, gradeColumnName):
        data = df.copy()
        #print(type(data[gradeColumnName]))
        data[gradeColumnName] = data[gradeColumnName].str.replace("\"",'')
        data[gradeColumnName] = data[gradeColumnName].str.replace("AP","N")
        for grade, val in self.gradeMap.items():
            #print(grade, val)
            data[gradeColumnName] = data[gradeColumnName].str.replace(grade, val)
        return data
    
    def createSubjectSubset(self, df, courseName):
        df1 = df.copy()
        df1 = df1[df1.CourseIndivName == courseName]
        return df1
        
    def masterDataBinSplitonTier(self, data, tierVal):
        dataset = data.copy()
        #create a new column for splitting as binary values, if it is the given tier, then will be labelled 1 , else 0
        dataset['TierBinSplitVal'] = (dataset.TierLevel == float(tierVal)).map({True:1 , False: 0})
        return dataset
    
    def masterDataBinSplitonColumn(self, data, columnName, columnVal):
        dataset = data.copy()
        #create a new column for splitting as binary values, if it is the given tier, then will be labelled 1 , else 0
        dataset[columnName+'BinSplitVal'] = (dataset[columnName] == columnVal).map({True:1 , False: 0})
        return dataset
    
    def masterDataBinSplitonGrade(self, data, grade):
        dataset = data.copy()
        dataset["CourseIndivGrade"] = pd.to_numeric(dataset['CourseIndivGrade'])
        #create a new col for 10th interval, if it is greater than or equal to given 10th percentage then we label as 1, else 0
        dataset['CourseGradeBinSplitVal'] = (dataset['CourseIndivGrade'] >= grade).map({True:1 , False: 0})
        return dataset
    
    def masterDataBinSpliton10th( self, data, perc10):
        dataset = data.copy()
        #create a new col for 10th interval, if it is greater than or equal to given 10th percentage then we label as 1, else 0
        dataset['10PercBinSplitVal'] = (dataset['10thPercentage'] >= float(perc10)).map({True:1 , False: 0})
        return dataset
    
    def masterDataBinSpliton12th( self, data, perc12):
        dataset = data.copy()
        #create a new col for 10th interval, if it is greater than or equal to given 10th percentage then we label as 1, else 0
        dataset['12PercBinSplitVal'] = (dataset['12thPercentage'] >= float(perc12)).map({True:1 , False: 0})
        return dataset
    
    def calc_entropy(self, column):
        """
        Calculate entropy given a pandas series, list, or numpy array.
        """
        # Compute the counts of each unique value in the column
        counts = np.bincount(column)
        # Divide by the total column length to get a probability
        probabilities = counts / len(column)

        # Initialize the entropy to 0
        entropy = 0
        # Loop through the probabilities, and add each one to the total entropy
        for prob in probabilities:
            if prob > 0:
                # use log from math and set base to 2
                entropy += prob * math.log(prob, 2)

        return -entropy
    
    def calc_information_gain(self, data, split_name, target_name):
        """
        Calculate information gain given a data set, column to split on, and target
        """
        # Calculate the original entropy
        original_entropy = self.calc_entropy(data[target_name])

        #Find the unique values in the column
        values = data[split_name].unique()
        #print(values)      
            
        # Make two subsets of the data, based on the unique values
        left_split = data[data[split_name] == values[0]]
        right_split = data[data[split_name] == values[1]]

        # Loop through the splits and calculate the subset entropies
        to_subtract = 0
        for subset in [left_split, right_split]:
            prob = (subset.shape[0] / data.shape[0]) 
            to_subtract += prob * self.calc_entropy(subset[target_name])

        # Return information gain
        return original_entropy - to_subtract

    def getEntropy10th(self, df, columnName, columnVal):
        #Now repeat the above process for CGPA by trying for each value differing by 0.1,
        #parallely see which threshold CGPA gives the max info gain(min entropy) and update the same
        #store the information gains for each CGPA threshold in infGainMap
        data = df.copy()
        data1 = self.masterDataBinSplitonColumn(data, columnName, columnVal)
        i = 60.0
        infGainMap = {}
        maxInfGain = [-1,-1]
        while(i<=100):
            #print(i)
            i = round(i,1)
            tempData = self.masterDataBinSpliton10th(data1, i)
            if(len(tempData['10PercBinSplitVal'].unique())<2):
                tempVal = i
                infGain = -100000
            else:
                infGain = self.calc_information_gain(tempData, '10PercBinSplitVal', columnName+'BinSplitVal')
            infGainMap[i] = infGain
            if(maxInfGain[1]< infGain):
                maxInfGain[0] = i
                maxInfGain[1] = infGain
            #print(infGain)
            #print("\n")
            i= i +1
        maxInf = maxInfGain[0]
        if(maxInfGain[0]==-1 and tempVal!=-1):
            maxInf = tempVal
        #map_grade = {"S": 7, "A":6, "B":5 ,"C":4, "D":3, "E":2, "F":1, "NA":-1}
        #listOfKeys = [key  for (key, value) in map_grade.items() if value == maxInf ]
        self.Optimal10thMarks = maxInf
        return maxInf
    
    def getEntropy12th(self, df, columnName, columnVal):
        #Now repeat the above process for CGPA by trying for each value differing by 0.1,
        #parallely see which threshold CGPA gives the max info gain(min entropy) and update the same
        #store the information gains for each CGPA threshold in infGainMap
        data = df.copy()
        data1 = self.masterDataBinSplitonColumn(data, columnName, columnVal)
        i = 60.0
        infGainMap = {}
        maxInfGain = [-1,-1]
        while(i<=100):
            #print(i)
            i = round(i,1)
            tempData = self.masterDataBinSpliton12th(data1, i)
            if(len(tempData['12PercBinSplitVal'].unique())<2):
                tempVal = i
                infGain = -100000
            else:
                infGain = self.calc_information_gain(tempData, '12PercBinSplitVal', columnName+'BinSplitVal')
            infGainMap[i] = infGain
            if(maxInfGain[1]< infGain):
                maxInfGain[0] = i
                maxInfGain[1] = infGain
            #print(infGain)
            #print("\n")
            i= i +1
            
        maxInf = maxInfGain[0]
        if(maxInfGain[0]==-1 and tempVal!=-1):
            maxInf = tempVal
        #map_grade = {"S": 7, "A":6, "B":5 ,"C":4, "D":3, "E":2, "F":1, "NA":-1}
        #listOfKeys = [key  for (key, value) in map_grade.items() if value == maxInf ]
        self.Optimal12thMarks = maxInf
        return maxInf

    def getEntropyGrade(self, df, courseName, columnName, columnVal):
        subSubset = self.createSubjectSubset(df, courseName)
        #split dataset on tier, n this case trying Tier1
        subSetSub = self.masterDataBinSplitonColumn(subSubset, columnName, columnVal )
        #parallely see which threshold Grade gives the max info gain(min entropy) and update the same
        #store the information gains for each Grade threshold in infGainMap
        tempVal = -1
        i = 2
        infGainMap = {}
        maxInfGain = [-1,-1]
        while(i<=7):
            #print(i)
            i = round(i,1)
            tempData = self.masterDataBinSplitonGrade(subSetSub, i)
            #print(tempData)
            if(len(tempData['CourseGradeBinSplitVal'].unique())<2):
                tempVal = i
                infGain = -100000
            else:
                infGain = self.calc_information_gain(tempData, 'CourseGradeBinSplitVal', columnName+'BinSplitVal')
            infGainMap[i] = infGain
            if(maxInfGain[1]< infGain):
                maxInfGain[0] = i
                maxInfGain[1] = infGain
            #print(infGain)
            #print("\n")
            i= i +1

        #print("\n\nMaximum information gain is found for : " + str(maxInfGain[0]) + " with value " + str(maxInfGain[1]))
        maxInf = maxInfGain[0]
        if(maxInfGain[0]==-1 and tempVal!=-1):
            maxInf = tempVal
        #map_grade = {"S": 7, "A":6, "B":5 ,"C":4, "D":3, "E":2, "F":1, "NA":-1}
        #listOfKeys = [key  for (key, value) in map_grade.items() if value == maxInf ]
        return maxInf
    
    def getOptimalGradeForAllCourses(self, data, columnName, columnVal, gradeColumnName, courseColumnName):
        df = data.copy()
        df = self.getSubsetNotNA(df, columnName)
        self.setMapGradeToNumber({})
        df1 = self.mapColumnGradeToNumber(df, gradeColumnName)
        df2 = self.CombineCourseGrade(df1, courseColumnName, gradeColumnName)
        #print(df2['CourseGradeCombo'])
        df3 = self.SplitCourseToIndividualRows(df2, columnName)
        df4 = self.preprocessCourseNameGradeIndiv(df3)
        #print(type(df4.CourseIndivGrade))
        listOfCourses = df4.CourseIndivName.unique()
        courseOptimalGrade = {}
        for course in listOfCourses:
            #print(course)
            grade = self.getEntropyGrade(df4, course, columnName, columnVal)
            courseOptimalGrade[course] = grade
        self.courseOptimalGrade = courseOptimalGrade
        df5 = self.addCourseGradeBasedItemset(df2, courseOptimalGrade, courseColumnName)
        return df5
    
    def addCourseGradeBasedItemset(self, df, courseOptimalGrade, courseColumnName):
        data = df.copy()
        #courseoptimalGrade = self.courseOptimalGrade
        #print(courseOptimalGrade)
        map_grade = {7: "S", 6:"A", 5:"B" ,4:"C", 3:"D", 2:"E", 1:"F", 0:"NA",-1:"NA"}
        #print(data.columns)
        data['ValidCourses']=""
        for idx, row in data.iterrows():
            validCourses = []
            for key in row['CourseGradeCombo']:
                row['CourseGradeCombo'][key] = int( row['CourseGradeCombo'][key])
                #print(keyPreProc)
                keyPreProc =key.strip().lower().replace("-i"," - i").replace("web technologies 1", "web technologies - i").replace("technology", "technologies").replace("web technologies- i", "web technologies - i").replace("web technologies ii", "web technologies - ii")
                #print(keyPreProc)
                if(keyPreProc in courseOptimalGrade):
                    if(row['CourseGradeCombo'][key]>= courseOptimalGrade[keyPreProc]):
                        validCourses.append(courseColumnName+"_"+keyPreProc+"::"+map_grade[courseOptimalGrade[keyPreProc]]+" and higher")
                else:
                    print("key not oresent", key)
            #print(validCourses)
            data.at[idx,'ValidCourses'] = validCourses
        self.validCoursesData = data[['USN','ValidCourses']]
        return data[['USN','ValidCourses']]
    
    def getOptimal10thMarks(self):
        return self.Optimal10thMarks
    
    def getOptimal10thMarks(self):
        return self.Optimal12thMarks
    
    def getValidCoursesItemset(self):
        return self.validCoursesData

In [149]:
path = 'combinedMasterLatest.csv'
master = pd.read_csv (path) 
obj = EntropyBasedItemset(master)
datatemp = obj.getOptimalGradeForAllCourses(master, 'TierLevel', '1', 'CourseGrade', 'CourseName')
Entropy10thVal = obj.getEntropy10th(master, 'TierLevel', '1')
Entropy12thVal = obj.getEntropy12th(master, 'TierLevel', '1')

In [152]:
master.columns

Index(['Unnamed: 0', 'USN', '10thPercentage', '12thPercentage', 'AwardsNLP',
       'Branch', 'CGPA', 'CTC', 'CoCurricularActivities', 'CompanyNameNLP',
       'CoreCourseGrade', 'CoreCourseName', 'CourseGrade', 'CourseName',
       'ElectiveCourseGrade', 'ElectiveCourseName', 'EmploymentType',
       'ExternalCertificatesDomain', 'ExternalCertificatesKey',
       'GeneralSkills', 'GitHubLink', 'InternshipCompany',
       'InternshipProjectDomain', 'LanguagesNLP', 'LinkedInLink',
       'MinorAttended', 'Name', 'NoofInternships', 'NoofProjects',
       'NumberOfOffers', 'OtherDetails', 'ProgLanguages',
       'ProjectDetailDomain', 'PublicationNLP', 'ResearchDomain',
       'ScholarshipsNLP', 'Stipend(K)', 'TierLevel', 'VolunteeringWork',
       'WorkshopsDomain', 'WorkshopsOrg', 'SoftwareTools', 'ProgLanguagesNLP'],
      dtype='object')